In [114]:
import base64
import time
import cv2
import json
import re
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

In [115]:
def intializeweb(url):
    now = str(datetime.now())
    child=now.replace(":","-")
    child=child.replace(".","-")
    parent = r"C:\\Users\\Jegadeesh\\Downloads\\Documents\\"
    path = os.path.join(parent, child)    
    try: 
        os.mkdir(path) 
    except OSError as error: 
        print(error)  
    chrome_options = webdriver.ChromeOptions()

    settings = {"recentDestinations": [{"id": "Save as PDF", "origin": "local", "account": ""}
                                      ], "selectedDestinationId": "Save as PDF", "version": 2}



    prefs = {
        'printing.print_preview_sticky_settings.appState': json.dumps(settings),
        'savefile.default_directory': f"{path}"
    }
    chrome_options.add_argument('--kiosk-printing')
    chrome_options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
    driver.get(url)
    return driver,path

def create_dataframe(list1,list2,split_into,header):
    split1=np.array_split(list1,split_into)
    split2=np.array_split(list2,split_into)
    df = pd.DataFrame()
    for i in range(0,split_into):
        key1=(" ")*i
        key2=header+(" ")*i
        a=pd.DataFrame({key1:split1[i].tolist()})
        b=pd.DataFrame({key2:split2[i].tolist()})
        df=pd.concat([df,a,b],axis=1)
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'}).hide_index()
    display(left_aligned_df)

def create_list(list_value):
    i=0
    results1=[]
    results2=[]
    for result in list_value:
        l1=str(i)
        l2=result.text
        results1.append(l1)
        results2.append(l2)
        i=i+1
    return results1,results2

def enter_details(driver):
    #Clicking Patta Chitta
    xpath="//strong[contains(text(),'பட்டா / சிட்டா விவரங்களை பார்வையிட')]"
    driver.find_element(by=By.XPATH, value=xpath).click()
    time.sleep(1)
    #districtcode
    xpath="//select[@id='districtCode']"
    dropdown=driver.find_element(by=By.XPATH, value=xpath)
    dd=Select(dropdown)
    all_available_options = dd.options
    print("மாவட்டம் அடிப்படையில் குறியீடு")
    results1,results2=create_list(all_available_options)
    create_dataframe(results1,results2,6,"மாவட்டம்")
    district=int(input("Enter the district index:"))
    dd.select_by_index(district)
    time.sleep(2)
    
    #taluk code
    xpath="//select[@name='talukCode']"
    dropdown=driver.find_element(by=By.XPATH, value=xpath)
    dd=Select(dropdown)
    all_available_options = dd.options
    print("வட்டம் அடிப்படையில் குறியீடு ")
    results1,results2=create_list(all_available_options)
    create_dataframe(results1,results2,4,"வட்டம்")
    taluk=int(input("Enter the taluk index:"))
    dd.select_by_index(taluk)
    time.sleep(2)
    
    #village code
    xpath="//select[@name='villageCode']"
    dropdown=driver.find_element(by=By.XPATH, value=xpath)
    dd=Select(dropdown)
    all_available_options = dd.options
    print("கிராமம் அடிப்படையில் குறியீடு ")
    results1,results2=create_list(all_available_options)
    create_dataframe(results1,results2,4,"கிராமம்")
    village=int(input("Enter the village index:"))
    dd.select_by_index(village)
    time.sleep(2)
    p=["1 பட்டா எண்","2 புல எண்","3 பெயர் வாரியான தேடல்"]
    for i in p:
        print(i,end="    ")
    value=int(input("Enter the index value you need:"))
    xpath=f"(//input[@id='viewOpt'])[{value}]"
    driver.find_element(by=By.XPATH, value=xpath).click()
    
    num=int(input("Number of files to be checked: "))
    survey_list=[]
    if value==1:
        for i in range(0,num):
            s=int(input("பட்டா எண்"))
            survey_list.append(s)
    elif value==2:
        for i in range(0,num):
            s=int(input("புல எண்: "))
            n=str(input("உட்பிரிவு எண்: ")).upper()
            z=[s,n]
            survey_list.append(z)
    else:
        print("Sorry automation not available")
        
    
    return district,taluk,village,value,survey_list

def automation(driver,district,taluk,village,value,survey_list,path):
    for i in range(0,len(survey_list)):
        try:
            if len(survey_list[i])>1:
                #districtcode
                xpath="//select[@id='districtCode']"
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_index(district)
                time.sleep(2)
                #taluk code
                xpath="//select[@name='talukCode']"
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_index(taluk)
                time.sleep(2)
                #village code
                xpath="//select[@name='villageCode']"
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_index(village)
                #patta or chitta
                xpath=f"(//input[@id='viewOpt'])[{value}]"
                driver.find_element(by=By.XPATH, value=xpath).click()
                xpath="//input[@id='surveyNo']"
                driver.find_element(by=By.XPATH, value=xpath).clear()
                driver.find_element(by=By.XPATH, value=xpath).send_keys(survey_list[i][0])
                xpath="//select[@name='subdivNo']"
                driver.find_element(by=By.XPATH, value=xpath).click()
                time.sleep(2)
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_value(survey_list[i][1])
                xpath="/html/body/div[1]/section/table/tbody/tr[2]/td/table/tbody/tr/td/div/form/div[25]/div[2]/img"
                ele_captcha =driver.find_element(by=By.XPATH, value=xpath)

                img_captcha_base64 = driver.execute_async_script("""
                        var ele = arguments[0], callback = arguments[1];
                        ele.addEventListener('load', function fn(){
                          ele.removeEventListener('load', fn, false);
                          var cnv = document.createElement('canvas');
                          cnv.width = this.width; cnv.height = this.height;
                          cnv.getContext('2d').drawImage(this, 0, 0);
                          callback(cnv.toDataURL('image/jpeg').substring(22));
                        }, false);
                        ele.dispatchEvent(new Event('load'));
                        """, ele_captcha)

                with open(r"captcha.jpg", 'wb') as f:
                        f.write(base64.b64decode(img_captcha_base64))
                img_base64 = driver.execute_script("""
                var ele = arguments[0];
                var cnv = document.createElement('canvas');
                cnv.width = 280; cnv.height = 70;
                cnv.getContext('2d').drawImage(ele, 0, 0);
                return cnv.toDataURL('image/jpeg').substring(22);    
                """, driver.find_element(by=By.XPATH, value=xpath))  #"/html/body/form/div[2]/div[3]/span[3]/div[1]/img"))
                with open(r"image.jpg", 'wb') as f:
                    f.write(base64.b64decode(img_base64)) 
                img = cv2.imread('image.jpg')
                captcha=pytesseract.image_to_string(img)
                xpath="//input[@id='captcha']"
                driver.find_element(by=By.XPATH, value=xpath).send_keys(captcha)
                xpath="//input[@value='சமர்ப்பி']"
                driver.find_element(by=By.XPATH, value=xpath).click()
                time.sleep(1)
                driver.execute_script('window.print();')  
                time.sleep(2)
                xpath="//a[normalize-space()='Back']"
                driver.find_element(by=By.XPATH, value=xpath).click()
                for filename in os.listdir(path):
                    if re.match('வட்டாட்சியர்', filename):
                        my_dest =str(survey_list[i][0]) + "-" +str(survey_list[i][1]) + ".pdf"
                        my_source =path +"\\"+ filename
                        my_dest =path +"\\"+ my_dest
                        # rename() function will
                        # rename all the files
                        os.rename(my_source, my_dest)
            else:
                #districtcode
                xpath="//select[@id='districtCode']"
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_index(district)
                time.sleep(2)
                #taluk code
                xpath="//select[@name='talukCode']"
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_index(taluk)
                time.sleep(2)
                #village code
                xpath="//select[@name='villageCode']"
                dropdown=driver.find_element(by=By.XPATH, value=xpath)
                dd=Select(dropdown)
                all_available_options = dd.options
                dd.select_by_index(village)
                time.sleep(2)
                #patta or chitta
                xpath=f"(//input[@id='viewOpt'])[{value}]"
                driver.find_element(by=By.XPATH, value=xpath).click()
                xpath="//input[@name='pattaNo']"
                driver.find_element(by=By.XPATH, value=xpath).clear()
                driver.find_element(by=By.XPATH, value=xpath).send_keys(survey_list[i])
                xpath="/html/body/div[1]/section/table/tbody/tr[2]/td/table/tbody/tr/td/div/form/div[25]/div[2]/img"
                ele_captcha =driver.find_element(by=By.XPATH, value=xpath)
                img_captcha_base64 = driver.execute_async_script("""
                        var ele = arguments[0], callback = arguments[1];
                        ele.addEventListener('load', function fn(){
                          ele.removeEventListener('load', fn, false);
                          var cnv = document.createElement('canvas');
                          cnv.width = this.width; cnv.height = this.height;
                          cnv.getContext('2d').drawImage(this, 0, 0);
                          callback(cnv.toDataURL('image/jpeg').substring(22));
                        }, false);
                        ele.dispatchEvent(new Event('load'));
                        """, ele_captcha)

                with open(r"captcha.jpg", 'wb') as f:
                        f.write(base64.b64decode(img_captcha_base64))
                img_base64 = driver.execute_script("""
                var ele = arguments[0];
                var cnv = document.createElement('canvas');
                cnv.width = 280; cnv.height = 70;
                cnv.getContext('2d').drawImage(ele, 0, 0);
                return cnv.toDataURL('image/jpeg').substring(22);    
                """, driver.find_element(by=By.XPATH, value=xpath))  #"/html/body/form/div[2]/div[3]/span[3]/div[1]/img"))
                with open(r"image.jpg", 'wb') as f:
                    f.write(base64.b64decode(img_base64)) 
                img = cv2.imread('image.jpg')
                captcha=pytesseract.image_to_string(img)
                xpath="//input[@id='captcha']"
                driver.find_element(by=By.XPATH, value=xpath).send_keys(captcha)
                xpath="//input[@value='சமர்ப்பி']"
                driver.find_element(by=By.XPATH, value=xpath).click()
                time.sleep(2)
                driver.execute_script('window.print();')  
                time.sleep(1)
                xpath="//a[normalize-space()='Back']"
                driver.find_element(by=By.XPATH, value=xpath).click()
                for filename in os.listdir(path):
                    if re.match('வட்டாட்சியர்', filename):
                        my_dest =str(survey_list[i][0]) + ".pdf"
                        my_source =path +"\\"+ filename
                        my_dest =path +"\\"+ my_dest
                        # rename() function will
                        # rename all the files
                        os.rename(my_source, my_dest)
            print(f"{survey_list[i]} downloaded")
        except:
            print(f"Exception occured skipping{survey_list[i]}")
            driver.refresh()
        
    print("Execution Completed")

In [355]:
url="https://eservices.tn.gov.in/eservicesnew/home.html"
driver,path=intializeweb(url)
district,taluk,village,value,survey_list=enter_details(driver)
automation(driver,district,taluk,village,value,survey_list,path)





====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\Jegadeesh\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
Driver [C:\Users\Jegadeesh\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


மாவட்டம் அடிப்படையில் குறியீடு


,மாவட்டம்,,மாவட்டம்,,மாவட்டம்,,மாவட்டம்,,மாவட்டம்,,மாவட்டம்
0,தேர்வு செய்க ...,7,திண்டுக்கல்,14,மதுரை,21,ராமநாதபுரம்,27,தேனி,33,திருப்பூர்
1,அரியலூர்,8,ஈரோடு,15,மயிலாடுதுறை,22,ராணிப்பேட்டை,28,திருச்சிராப்பள்ளி,34,திருவள்ளூர்
2,செங்கல்பட்டு,9,கள்ளக்குறிச்சி,16,நாகப்பட்டினம்,23,சேலம்,29,தி௫வாரூர்,35,திருவண்ணாமலை
3,சென்னை,10,காஞ்சிபுரம்,17,நாமக்கல்,24,சிவகங்கை,30,தூத்துக்குடி,36,வேலூர்
4,கோயம்புத்தூர்,11,கன்னியாகுமரி,18,நீலகிரி,25,தென்காசி,31,திருநெல்வேலி,37,விழுப்புரம்
5,கடலூர்,12,கரூர்,19,பெரம்பலூர்,26,தஞ்சாவூர்,32,திருப்பத்தூர்,38,விருதுநகர்
6,தர்மபுரி,13,கிருஷ்ணகிரி,20,புதுக்கோட்டை,nan,nan,nan,nan,nan,nan


Enter the district index:12
வட்டம் அடிப்படையில் குறியீடு 


,வட்டம்,,வட்டம்,,வட்டம்,,வட்டம்
0,தேர்வு செய்க ...,2,கடவூர்,4,கிருஷ்ணராயபுரம்,6,மண்மங்கலம்
1,அரவக்குறிச்சி,3,கரூர்,5,குளித்தலை,7,புகளூர்


Enter the taluk index:5
கிராமம் அடிப்படையில் குறியீடு 


,கிராமம்,,கிராமம்,,கிராமம்,,கிராமம்
0,தேர்வு செய்க ...,12,குமாரமங்கலம்,24,நங்கவரம் தெற்கு - 2,35,இராஜேந்திரம் தெற்கு
1,ஆலத்தூர்,13,மணத்தட்டை,25,நெய்தலூர் வடக்கு,36,சத்தியமங்கலம்
2,சின்னியம்பாளையம்,14,மருதூர் வடக்கு - 1,26,நெய்தலூர் தெற்கு,37,சேப்பளாபட்டி
3,இரண்யமங்கலம்,15,மருதூர் வடக்கு - 2,27,பாதிரிபட்டி,38,சூரியனூர்
4,கூடலூர்,16,மருதூர் தெற்கு - 1,28,பில்லூர்,39,தளிஞ்சி
5,இனுங்கூர்,17,மருதூர் தெற்கு - 2,29,பொருந்தலூர்,40,தோகமலை
6,கடம்பர்கோவில்,18,முதலைபட்டி,30,பொய்யாமணி,41,ஊத்துபட்டி
7,கல்லடை,19,நாகனூர்,31,புழுதேரி,42,வடசேரி
8,கள்ளை,20,நல்லூர்,32,புத்தூர்,43,வைகநல்லூர் வடக்கு
9,கழுகூர்,21,நங்கவரம் வடக்கு - 1,33,இராச்சாண்டார்திருமலை,44,வைகநல்லூர் தெற்கு


Enter the village index:7
1 பட்டா எண்    2 புல எண்    3 பெயர் வாரியான தேடல்    Enter the index value you need:2
Number of files to be checked: 2
புல எண்: 180
உட்பிரிவு எண்: 4b
புல எண்: 180
உட்பிரிவு எண்: 8b
[180, '4B'] downloaded
[180, '8B'] downloaded
Execution Completed
